# Persian News Classification
In this notebook, I tried to categorize BBC Persian news using the title. With help of  hazm library for preprocessing the texts and sklearn for train model.

## Overview 
### 1) Understand the data (Shape , missing values  , ...)
### 2) Data Preprocessing 
### 4) Model Building 
### 5) Visualization Result

In [1]:
# Read Data
import pandas as pd
import numpy as np


# Train
from sklearn.naive_bayes import MultinomialNB
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow as tf

# Preprocessing
from sklearn.model_selection import train_test_split
from stopwords_guilannlp import stopwords_output
from hazm import *
import re

# Visualization
from sklearn.metrics import accuracy_score , classification_report
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
df = pd.read_json('bbcpersian.json')

In [3]:
df.head()

,category,link,title,publish_time,related_topics,body
0,business,https://www.bbc.com/persian/business-53642810,تهران و دوبی؛ ملاقات مجازی، منافع حقیقی,۱۳ مرداد ۱۳۹۹ - ۳ اوت ۲۰۲۰,"[جامعه ایران, روابط خارجی ایران, عربستان سعودی...",برای پخش این فایل لطفا جاوا اسکریپت را فعال یا...
1,world,https://www.bbc.com/persian/world-58579904,نظرسنجی حاکی از اضطراب نسل جوان درباره تغییر ا...,۲۵ شهریور ۱۴۰۰ - ۱۶ سپتامبر ۲۰۲۱,"[تغییرات آب و هوایی, علمی]",\n\n\nیک نظرسنجی تازه جهانی نشان دهنده عمق اضط...
2,arts,https://www.bbc.com/persian/arts-54913866,جفری توبین، خبرنگار ارشد نیویورکر برای عورت‌نم...,۲۲ آبان ۱۳۹۹ - ۱۲ نوامبر ۲۰۲۰,"[میراث فرهنگی, ایالات متحده آمریکا]",\n\n\nمجله فرهنگی نیویورکر جفری توبین، یکی از ...
3,sport,https://www.bbc.com/persian/sport-52083137,از جنگل سیاه تا لیورپول؛ ماجرای سفر رویایی یور...,۱۰ فروردین ۱۳۹۹ - ۲۹ مارس ۲۰۲۰,"[فوتبال, آلمان, ورزش]",\nسم شرینگهامبی‌بی‌سی\n\n\nینس هاس هنوز اولین ...
4,arts,https://www.bbc.com/persian/arts-56242134,گفتگو با شیدا بازیار؛ انقلاب و ۴۰ سال بعد از آ...,۱۱ اسفند ۱۳۹۹ - ۱ مارس ۲۰۲۱,"[کتاب و ادبیات, ادبیات جهان, ایران]",برای پخش این فایل لطفا جاوا اسکریپت را فعال یا...


In [4]:
df.isnull().sum()

category          0
link              0
title             0
publish_time      0
related_topics    0
body              0
dtype: int64

In [5]:
df.category.value_counts()

iran                   754
sport                  735
world                  714
business               372
arts                   359
science                258
afghanistan            232
magazine               107
iran-features          100
world-features          80
blog-viewpoints         29
vert-earth               3
vert-fut                 3
vert-cul                 3
blogs                    2
interactivity            2
58182718                 1
58092745                 1
58008330                 1
institutional            1
56026690                 1
59033766                 1
53817930                 1
57938483                 1
59130485                 1
58955332                 1
58168992                 1
58850860                 1
57816007                 1
57816004                 1
58421548                 1
57213241                 1
58895066                 1
afghan-press-review      1
58885163                 1
57959492                 1
58813922                 1
5

In [6]:
# Select balance data  from whole data 
df = df.loc[df['category'].isin(['iran','sport','world'])]

In [7]:
puncs = ['،', '.', ',', ':', ';', '"',"'",'/','\\','_','-']

normalizer = Normalizer()

# Create custom stop_word 
def load_stopwords():
    f = open("fa_stop_words.txt", "r", encoding='utf8')
    stopwords = f.read()
    stopwords = stopwords.split('\n')
    stopwords = set(stopwords)
    custom_stop_words = {'آنكه','آيا','بدين','براين','بنابر','میشه','میکنه','باشه','سلام','میکشه','اونی','و','در','از','چه'}
    stopwords = stopwords  | custom_stop_words
    stopwords = list(stopwords)[1:]
    unwanted_num = {'خوش','بهتر','بد','خوب','نیستم','عالی','نیست','فوق','بهترین','خیلی', 'نبود'} 
    stopwords = [ele for ele in stopwords if ele not in unwanted_num]
    return stopwords


def clean_doc(doc):
    doc = normalizer.normalize(doc) # Normalize document using Hazm Normalizer
    tokenized = word_tokenize(doc)  # Tokenize text
    tokens = []
    for t in tokenized:
      temp = t
      for p in puncs:
        temp = temp.replace(p, '')
      tokens.append(temp)
    stop_set = load_stopwords()
    tokens = [w for w in tokens if not w in stop_set] # Remove stop words
    
    tokens = [w for w in tokens if not len(w) <= 1]  # Clean words
    tokens = [w for w in tokens if not w.isdigit()] #  Remove digits 
    tokens =  [w for w in tokens if not re.match(r'[A-Z]+', w, re.I)] # Remove english letters 
    tokens = ' '.join(tokens)
    return tokens

In [8]:
df.title = df['title'].apply(lambda x :clean_doc(x)) 

In [9]:
df.head()

,category,link,title,publish_time,related_topics,body
1,world,https://www.bbc.com/persian/world-58579904,نظرسنجی حاکی اضطراب نسل جوان تغییر اقلیم,۲۵ شهریور ۱۴۰۰ - ۱۶ سپتامبر ۲۰۲۱,"[تغییرات آب و هوایی, علمی]",\n\n\nیک نظرسنجی تازه جهانی نشان دهنده عمق اضط...
3,sport,https://www.bbc.com/persian/sport-52083137,جنگل سیاه لیورپول ماجرای سفر رویایی یورگن کلوپ,۱۰ فروردین ۱۳۹۹ - ۲۹ مارس ۲۰۲۰,"[فوتبال, آلمان, ورزش]",\nسم شرینگهامبی‌بی‌سی\n\n\nینس هاس هنوز اولین ...
6,world,https://www.bbc.com/persian/world-55116455,نقض حقوق زنان آزمون بکارت بریتانیا ممنوع,۸ آذر ۱۳۹۹ - ۲۸ نوامبر ۲۰۲۰,"[بریتانیا, زنان, سلامت زنان, حقوق زنان]",برای پخش این فایل لطفا جاوا اسکریپت را فعال یا...
9,world,https://www.bbc.com/persian/world-58596292,بحران انسانی مرز مکزیک آمریکا تجمع پناهجو پل مرزی,۲۶ شهریور ۱۴۰۰ - ۱۷ سپتامبر ۲۰۲۱,"[پناهندگان و پناهجویان, ایالات متحده آمریکا]",\n\n\nجمع شدن حدود ۱۰ هزار پناهجو در اطراف پل ...
10,sport,https://www.bbc.com/persian/sport-58071798,المپیک توکیو محمد هادی ساروی کشتی فرنگی مدال برنز,۱۲ مرداد ۱۴۰۰ - ۳ اوت ۲۰۲۱,"[کشتی, ورزش ایران, المپیک, ورزش]",\n\n\nمحمدهادی ساروی، با پیروزی در دیدار رده‌ب...


In [10]:
# extract related_topics  
related1 = df['related_topics'].apply(lambda x : x[0])
related2 = df['related_topics'].apply(lambda x : x[1])

In [11]:
# include related_topics with title 
X = related1 + ' ' + related2 + ' ' + df.title 


label_mapping = {'iran':0,'sport':1,'world':2}
df = df.copy()
df['category'] = df['category'].replace(label_mapping)

y = df['category'].copy()

In [12]:
X.shape

(2203,)

In [13]:
y.shape

(2203,)

In [15]:
tf=TfidfVectorizer()
  
x=tf.fit_transform(X) 

In [16]:
x.shape

(2203, 4674)

In [17]:
# Split train and test set
X_train , X_test , y_train , y_test = train_test_split(x,y,test_size=0.1)

In [18]:
nb = MultinomialNB()

# Model
nb_model = nb.fit(X_train, y_train)

# Predict
nb_predict = nb.predict(X_test)

# Accuracy
nb_acc = accuracy_score(y_test,nb_predict)

print('nb test accuracy:', nb_acc)

nb test accuracy: 0.918552036199095


In [19]:
cr = classification_report(y_test, nb_predict)

In [20]:
print(cr)

              precision    recall  f1-score   support

           0       0.90      0.95      0.92        74
           1       0.96      0.96      0.96        82
           2       0.89      0.83      0.86        65

    accuracy                           0.92       221
   macro avg       0.92      0.91      0.91       221
weighted avg       0.92      0.92      0.92       221



In [21]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

# Model
rf_model = rf.fit(X_train, y_train)

# Predict
rf_predict = rf.predict(X_test)

# Accuracy
nb_acc = accuracy_score(y_test,rf_predict)

print('nb test accuracy:', nb_acc)


nb test accuracy: 0.9502262443438914


In [22]:
cr = classification_report(y_test, rf_predict)

In [23]:
print(cr)

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        74
           1       1.00      0.98      0.99        82
           2       0.90      0.94      0.92        65

    accuracy                           0.95       221
   macro avg       0.95      0.95      0.95       221
weighted avg       0.95      0.95      0.95       221



In [26]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train,y_train)

gbc_predict = gbc.predict(X_test)

gbc_acc = accuracy_score(y_test,gbc_predict)

print('nb test accuracy:', gbc_acc)

nb test accuracy: 0.9592760180995475


In [29]:
cr = classification_report(y_test, gbc_predict)

In [30]:
print(cr)

              precision    recall  f1-score   support

           0       0.96      0.93      0.95        74
           1       1.00      0.99      0.99        82
           2       0.91      0.95      0.93        65

    accuracy                           0.96       221
   macro avg       0.96      0.96      0.96       221
weighted avg       0.96      0.96      0.96       221

